In [1]:
import urllib #For downloading webpages
import bs4
from bs4 import BeautifulSoup # For Parsing HTML Documents
import requests
import re
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd



In [2]:
%%time

#Create lists to save data--------------------------------------------
title = list()
year = list()
certi = list()
time = list()
release_date = list()
rating = list()
metascore = list()
budget = list()
Opening_Weekend_USA = list()
Gross_USA = list()
Cumulative_Worldwide_Gross = list()

# Read list of movies to be scraped
movie_list = pd.read_csv("C:/Users/neilr/OneDrive/Desktop/Notes/Matt 891/Matt_imdb project/IMDB_list.csv")
url_list = movie_list['movie_imdb_link'].to_list()

#Itterate through each movie to scrape data
for url in url_list:
    #Read The url---------------------------------------------------------
    try:
        page = requests.get(url)
    except:
        continue
        
    if page.status_code != 200:
        continue
        
    soup = BeautifulSoup(page.content, 'html.parser')

    #Get Basic Data-------------------------------------------------------
    Basicdata = list()
    
    
    try:
        # Get the whole basic data and split it in different attributes.
        t = soup.h1.text
        y = soup.h1.span.a.text
        a = soup.find('div', class_="subtext").text
        q = a.replace("   ","")
        q = q.replace("  ","")
        q = q.replace(",","")
        li = q.split('\n')

        Basicdata.extend([t,y,li[1],li[3],li[-2]])

        title.append(Basicdata[0])
        year.append(Basicdata[1])
        certi.append(Basicdata[2])
        time.append(Basicdata[3])
        release_date.append(Basicdata[4])
    except:
        title.append("-")
        year.append("-")
        certi.append("-")
        time.append("-")
        release_date.append("-")
        
    #Get Rating and MetaScore----------------------------------------------
    try:
        r = soup.find('div', class_="ratingValue").span.text
        rating.append(r)
    except:
        rating.append("-")
        
    flag = 0
    try:
        m = soup.find('div', class_="metacriticScore score_favorable titleReviewBarSubItem").span.text
        metascore.append(m)
        flag = 1
    except:    
        try:
            m = soup.find('div', class_="metacriticScore score_mixed titleReviewBarSubItem").span.text
            metascore.append(m)
        except:
            metascore.append("-")

    #Get the financials----------------------------------------------------
    count = [0,0,0,0]
    for h4 in soup.find_all('h4'):
        try:
            if "Budget:" in h4:
                count[0] = count[0] + 1
                budget.append(h4.next_sibling.strip())
        except:
            budget.append("-")
            
        try:
            if "Opening Weekend USA:" in h4:
                count[1] = count[1] + 1
                Opening_Weekend_USA.append(h4.next_sibling.strip())
        except:
            Opening_Weekend_USA.append("-")

        try:
            if "Gross USA:" in h4:
                count[2] = count[2] + 1
                Gross_USA.append(h4.next_sibling.strip())
        except:
            Gross_USA.append("-")
            
        try:
            if "Cumulative Worldwide Gross:" in h4:
                count[3] = count[3] + 1
                Cumulative_Worldwide_Gross.append(h4.next_sibling.strip())
        except:
            Cumulative_Worldwide_Gross.append("-")
            
    if count[0] == 0:
        budget.append("-")
    if count[1] == 0:
        Opening_Weekend_USA.append("-")
    if count[2] == 0:
        Gross_USA.append("-")
    if count[3] == 0:
        Cumulative_Worldwide_Gross.append("-")

# Store the whole data in a DataFrame
df = pd.DataFrame(list(zip(title,year,certi,time,release_date,rating,metascore,budget,Opening_Weekend_USA,
                       Gross_USA,Cumulative_Worldwide_Gross)), columns =['title','year','certi','time','release_date',
                                                                         'rating','metascore','budget','Opening_Weekend_USA',
                       'Gross_USA','Cumulative_Worldwide_Gross'])

Wall time: 35min


In [3]:
#Wall time: 35min

In [4]:
df.to_csv(r'C:\Users\neilr\OneDrive\Desktop\Notes\Matt 891\Matt_imdb project\Web_Scrape_data.csv',index = False,header=True)

# ---------------------------END---------------------------